### Junyu Sui, JS6130

## Q1
### 1.1

\begin{aligned}
&\alpha_t=a\left(R_t-R_T\right)+b\left(W_t-W_T\right)\\
&\begin{gathered}
&b=\rho_2 \sqrt{1-\rho_2^2}
&\rho_2 =0.3\\
&a=\frac{\rho b}{\sqrt{1-\rho^2}}, \forall \rho
\end{gathered}
\end{aligned}

In [1]:
\l /Users/suijunyu/Desktop/5440/columbiaHdb

time: 2019.01.03
raw: select from bin10 where date = time
raw: update reverse fills reverse date,
            reverse fills reverse mid,
            reverse fills reverse spread,
            reverse fills reverse vol,
            reverse fills reverse adv
            by id from raw;
\S 69 / set a random seed 
/ compute the return
raw: update R: (neg 1) + (last mid) % mid by id from raw
/ compute the volatility of return
deltas0:{first[x]-':x}
raw: update vol_R: sqrt var deltas0 R by id from raw
/compute the W(wiener process)
u12 : {[ n ] -6f + sum n cut (12* n ) ?1f }
raw: update W: vol_R * (0, sums u12[(count vol_R) - 1])  by id from raw
/duplicate the tbl with different rho, strat name
raw: raw cross ([]rho: 0, .05, .3; []name: 0, 1, 2)
/create alpha based on the formula
createAlpha:{[rho; R; W]
            b: 0.3 * sqrt(1 - 0.09);
            a: (rho * b) % sqrt(1 - rho * rho);
            alpha: (a * (R - last R)) + (b * (W - last W));
            :alpha}

tbl: update alpha: createAlpha[rho;R; W] by id, rho from raw

In [2]:
select time, id, R, W, rho, name, alpha from tbl

time     id R           W             rho  name alpha       
------------------------------------------------------------
09:30:00 0  -0.02958659 0             0    0    -0.004360113
09:30:00 0  -0.02958659 0             0.05 1    -0.004784   
09:30:00 0  -0.02958659 0             0.3  2    -0.007022906
09:30:10 0  -0.02971043 0.0003646641  0    0    -0.004255752
09:30:10 0  -0.02971043 0.0003646641  0.05 1    -0.004681414
09:30:10 0  -0.02971043 0.0003646641  0.3  2    -0.006929691
09:30:20 0  -0.02824664 -1.006247e-05 0    0    -0.004362992
09:30:20 0  -0.02824664 -1.006247e-05 0.05 1    -0.004767682
09:30:20 0  -0.02824664 -1.006247e-05 0.3  2    -0.00690519 
09:30:30 0  -0.02896681 1.652537e-06  0    0    -0.00435964 
09:30:30 0  -0.02896681 1.652537e-06  0.05 1    -0.004774647
09:30:30 0  -0.02896681 1.652537e-06  0.3  2    -0.006966653
09:30:40 0  -0.02839575 2.751703e-05  0    0    -0.004352238
09:30:40 0  -0.02839575 2.751703e-05  0.05 1    -0.004759064
09:30:40 0  -0.02839575 

### 1.2

The target impact state is given by:
$$
I_t=\frac{1}{2}\left(\alpha_t-\beta^{-1} \alpha_t^{\prime}\right)
$$

The equation of dQ is :

\begin{aligned}
&d Q=\frac{a d v}{0.8 v o l}\left(\beta I_t d t+d I_t\right)=\frac{1}{\lambda}\left(\beta I_t d t+d I_t\right)\\
&I_t^{-}=e^{-\beta *dt} I_{t-1}\\
&d Q_t=\frac{1}{\lambda}\left(I_t-I_{t-}\right)
\end{aligned}

In [43]:
/update the alpha prime
tbl: update alpha_2: 0^((alpha - xprev[60; alpha]) % 10) by id, rho from tbl

/lambda is 8 and h is 60
h: 60
beta: (log 2) % h
dt:1%6; / dt = 1/6 min
/update Impact and I-
computeImpact: {[alpha; alpha_2] 
                :0.5 * (alpha - alpha_2 % beta)}
tbl_impact: update I: computeImpact[alpha; alpha_2] by id, rho from tbl
tbl_impact: update I_p:  (1 xprev I) * (exp neg beta*dt) by id, rho from tbl_impact

computeTrade: {[adv; vol; I; I_p]
                dv: vol * sqrt 2340;
                lam: 8 * dv % adv;
                : (I - I_p) % lam}

tbl_dQ: update dQ: computeTrade[adv; vol; I; I_p] by id, rho from tbl_impact

select  dQ by id, rho from tbl_dQ

id rho | dQ                                                                  ..
-------| --------------------------------------------------------------------..
0  0   |  31180.55  -37500.53 -1637.383 -319.7112 -72631.71 38157.12  18785.9..
0  0.05|  30339.22  -30953.07 -5242.451 2079.042  -71960.49 38946.36  17146.6..
0  0.3 |  25895.41  3629.593  -24283.85 14748.87  -68415.21 43115.02  8488.28..
1  0   |  -2592.96  -32360.77 7269.736  -718.0847 -8553.39  8082.491  7985.64..
1  0.05|  811.3753  -32383.08 8302.964  -1996.142 -7642.206 8794.184  10046.7..
1  0.3 |  18792.54  -32500.94 13760.32  -8746.639 -2829.475 12553.24  20933.1..
2  0   |  -861.8352 604.4058  -10582.17 747.8175  13.49982  -1055.137 2674.06..
2  0.05|  -883.2312 583.0098  -10603.57 726.4216  -7.896151 -1076.533 2652.67..
2  0.3 |  -996.2413 469.9997  -10716.58 613.4115  -120.9063 -1189.543 2539.66..
3  0   |  -2091.29  -1959.907 -3267.49  -5653.909 6323.162  -9277.454 2629.28..
3  0.05|  -2083.699 -1621.963 -3479.556 

In [4]:
/show the ratio and first alpha
tbl_pic: select ratio: sum dQ % last adv, first alpha by id, name from tbl_dQ
tbl_pic:0!tbl_pic
tbl_pic

id name ratio       alpha       
--------------------------------
0  0    -0.06331212 -0.004360113
0  1    -0.06559829 -0.004784   
0  2    -0.07767348 -0.007022906
1  0    -0.05619257 -0.004225025
1  1    -0.0597229  -0.004665212
1  2    -0.07836959 -0.006990209
2  0    0.0388927   0.008589024 
2  1    0.03711505  0.008289345 
2  2    0.02772579  0.006706489 
3  0    0.06784427  0.01235294  
3  1    0.0661831   0.01247244  
3  2    0.05740907  0.01310366  
4  0    0.04856568  0.0007707228
4  1    0.04751007  0.0008427279
4  2    0.04193451  0.001223047 
5  0    0.06527045  0.0007910923
5  1    0.0637951   0.0007841308
5  2    0.05600254  0.0007473614
6  0    0.01315992  -0.001177255
6  1    0.01177446  -0.001283126
..


### 1.3

In [5]:
/ABtest function
abTest:{[tbl; strat1; strat2; prob1]
    /update the indicator which has prob1 to be 0 and 1-prob1 to be Yes 1
    tbl2: update indicator: 9h$prob1 < 1?1f by id, date from tbl;
    / Selected_strat means: p=prob1 is strat1 and p=1-prob1 is strat2
    tbl2: update selected_strat: ((1-indicator) * strat1) + indicator*strat2 by id,date from tbl2;
    / The table selects one strategy for each pair
    tbl_selected: select from tbl2 where name = selected_strat;
    tbl_selected: xcols [`date`time`id`selected_strat`indicator; tbl_selected]
    }
/here strat1 is name = 2, strat2 is name =0 and prob is 0.8
tbl2: abTest[tbl_dQ; 2; 0; 0.8]
tbl2

date       time     id selected_strat indicator trade     mid      spread    ..
-----------------------------------------------------------------------------..
2019.01.03 09:30:00 0  2              0         454071.8  91.715   0.00114871..
2019.01.03 09:30:10 0  2              0         -171833.3 91.72671 0.00107213..
2019.01.03 09:30:20 0  2              0         -301.2193 91.58853 7.658112e-..
2019.01.03 09:30:30 0  2              0         -78884.65 91.65646 0.00061264..
2019.01.03 09:30:40 0  2              0         -24705.54 91.60259 0.00084239..
2019.01.03 09:30:50 0  2              0         -92166.19 91.58385 0.00058712..
2019.01.03 09:31:00 0  2              0         -22823.01 91.56276 0.00066370..
2019.01.03 09:31:10 0  2              0         -4940.182 91.59088 0.00056159..
2019.01.03 09:31:20 0  2              0         -41691.88 91.59322 0.00053606..
2019.01.03 09:31:30 0  2              0         -539826.4 91.60727 0.00074028..
2019.01.03 09:31:40 0  2              0 

In [6]:
/we can see that the probability of select strat1 is almost 80%
num2: count select from tbl2 where selected_strat = 2
num0: count select from tbl2 where selected_strat = 0
num2 % (num2+num0) / almost 80%

0.760479


### 1.4

In [7]:
/use xrank to split into three parts based on their vol in order
/the group with smaller vol is bucket 0, medium is bucket 1, and large is bucket2
tbl_bucket: update bucket: 3 xrank vol from tbl_dQ
select first bucket by id from tbl_bucket

id| bucket
--| ------
0 | 0     
1 | 0     
2 | 1     
3 | 1     
4 | 0     
5 | 1     
6 | 1     
7 | 0     
8 | 2     
9 | 2     
10| 2     
11| 0     
12| 0     
13| 1     
14| 1     
15| 0     
16| 2     
17| 2     
18| 2     
19| 0     
..


In [8]:
tbl_bucket_low: select from tbl_bucket where bucket = 0
tbl_bucket_med: select from tbl_bucket where bucket = 1
tbl_bucket_high:select from tbl_bucket where bucket = 2

tbl_bucket_low2: abTest[tbl_bucket_low; 2; 0; 0.8]
tbl_bucket_med2: abTest[tbl_bucket_med; 2; 0; 0.8]
tbl_bucket_high2:abTest[tbl_bucket_high; 2; 0; 0.8]

num2: count select from tbl_bucket_low2 where selected_strat = 2
num0: count select from tbl_bucket_low2 where selected_strat = 0
num2 % (num2+num0) / almost 80%

num2: count select from tbl_bucket_med2 where selected_strat = 2
num0: count select from tbl_bucket_med2 where selected_strat = 0
num2 % (num2+num0) / almost 80%

num2: count select from tbl_bucket_high2 where selected_strat = 2
num0: count select from tbl_bucket_high2 where selected_strat = 0
num2 % (num2+num0) / almost 80%

0.8323353


0.7844311


0.8023952


We can see for three bucket, their ratios are almost 80%

## Q2
### 2.1

PnL formula for each time take is:
$$
\left(\frac{S_{t_n}}{S_{t_{n-1}}}-1\right) Q_{t_{n-1}}-\frac{1}{2}\left(I_{t_{n-1-}}+I_{t_{n-1}}\right) \Delta Q_{t_{n-1}}
$$

In [44]:
/ Define the function for PnL calculation
prob: 0.8
h: 60
beta: (log 2) % h
dt:1%6; / dt = 1/6 min

computePnL:{[tbl] /set tbl as input
    / Calculate I decay
    tbl: update PnL_f: ((neg 1) + mid % (1 xprev mid)) * 1 xprev sums dQ,
    PnL_IC: 0.5 * 1 xprev ( I + (1 xprev I) * exp neg beta*dt) * dQ by id from tbl;
    tbl: update PnL: PnL_f - PnL_IC from tbl;
    tbl: xcols [`date`time`id`PnL`dQ;tbl];
    tbl
    }

/compute the sample PnL with date = 2019.01.03
PnL: computePnL[tbl2]
PnL

date       time     id PnL       dQ        selected_strat indicator trade    ..
-----------------------------------------------------------------------------..
2019.01.03 09:30:00 0                      2              0         454071.8 ..
2019.01.03 09:30:10 0            25895.41  2              0         -171833.3..
2019.01.03 09:30:20 0  51.23226  3629.593  2              0         -301.2193..
2019.01.03 09:30:30 0  34.43902  -24283.85 2              0         -78884.65..
2019.01.03 09:30:40 0  -87.2153  14748.87  2              0         -24705.54..
2019.01.03 09:30:50 0  47.0191   -68415.21 2              0         -92166.19..
2019.01.03 09:31:00 0  -228.299  43115.02  2              0         -22823.01..
2019.01.03 09:31:10 0  149.8138  8488.288  2              0         -4940.182..
2019.01.03 09:31:20 0  29.50307  31202.43  2              0         -41691.88..
2019.01.03 09:31:30 0  112.268   -12106.76 2              0         -539826.4..
2019.01.03 09:31:40 0  -55.4977  52323.3

In [10]:
/show PnL of each stock and there selected strategy
PnL_id: select sum PnL, first selected_strat by id from PnL
PnL_id

id| PnL       selected_strat
--| ------------------------
0 | -49488.16 2             
1 | 9801.962  0             
2 | -10841.65 0             
3 | -13523.36 0             
4 | -5993.29  0             
5 | -9236.726 2             
6 | -20684.62 2             
7 | -5746.788 0             
8 | -103157.6 2             
9 | -33243.68 2             
10| -23199.74 0             
11| -13568.54 2             
12| -16038.89 2             
13| -3060     0             
14| -510649.5 2             
15| -16005.87 2             
16| -197557.8 2             
17| -15307.69 2             
18| -4368.332 2             
19| -97401.77 2             
..


In [11]:
/show the average PnL of each strategy
PnL_avg: select average_PnL: avg PnL by selected_strat from PnL_id
PnL_avg

selected_strat| average_PnL
--------------| -----------
0             | -16808.72  
2             | -23561.84  


Define the function for each day, the input is date

In [12]:
h: 60
beta: (log 2) % h
dt:1%6; / dt = 1/6 min
prob1: 0.8
strat1: 2
strat2: 0
\S 69 / set a random seed 
/combine the above process together to create computeFull function
computeFull:{[d] /here date is the input
            raw: select from bin10 where date = d;
            raw: update reverse fills reverse date,
                reverse fills reverse mid,
                reverse fills reverse spread,
                reverse fills reverse vol,
                reverse fills reverse adv
                by id from raw;

            raw: update R: (neg 1) + (last mid) % mid by id from raw;
            raw: update vol_R: sqrt var deltas0 R by id from raw;
            raw: update W: vol_R * (0, sums u12[(count vol_R) - 1])  by id from raw;
            raw: raw cross ([]rho: 0, .05, .3; []name: 0, 1, 2);

            tbl: update alpha: createAlpha[rho;R; W] by id, rho from raw;
            tbl: update alpha_2: 0^((alpha - xprev[60; alpha]) % 10) by id, rho from tbl;

            tbl_impact: update I: computeImpact[alpha; alpha_2] by id, rho from tbl;
            tbl_impact: update I_p:  (1 xprev I) * (exp neg beta*dt) by id, rho from tbl_impact;

            tbl_dQ: update dQ: computeTrade[adv; vol; I; I_p] by id, rho from tbl_impact;

            tbl2: abTest[tbl_dQ; strat1; strat2; prob1];
            PnL_sec: computePnL[tbl2];
            PnL: 0!select first date, daily_PnL: sum PnL, first adv, first selected_strat by id from PnL_sec;
            PnL}

tbl3: computeFull[2019.01.03]
tbl3

id date       daily_PnL adv          selected_strat
---------------------------------------------------
0  2019.01.03 -49488.16 1.122415e+08 2             
1  2019.01.03 9801.962  3.607207e+07 0             
2  2019.01.03 -10841.65 1.38682e+07  0             
3  2019.01.03 -13523.36 1.192783e+07 0             
4  2019.01.03 -5993.29  2.019054e+07 0             
5  2019.01.03 -9236.726 7931115      2             
6  2019.01.03 -20684.62 1.790894e+07 2             
7  2019.01.03 -5746.788 9922401      0             
8  2019.01.03 -103157.6 3.452761e+07 2             
9  2019.01.03 -33243.68 1.862319e+07 2             
10 2019.01.03 -23199.74 8.626539e+07 0             
11 2019.01.03 -13568.54 5.807214e+07 2             
12 2019.01.03 -16038.89 2.996236e+07 2             
13 2019.01.03 -3060     1.538175e+07 0             
14 2019.01.03 -510649.5 9.883369e+08 2             
15 2019.01.03 -16005.87 7.305665e+07 2             
16 2019.01.03 -197557.8 1.933401e+08 2             
17 2019.01.0

### The speed of running full year data is too slow, so here I just run first 60 days(about two months) to show my result

In [13]:
\S 69 / set a random seed 
/ read all dates and select the first 60 dates
date_list: "D"$system "ls ../columbiaHdb"
tbl_full: raze computeFull each date_list til 60
tbl_full

id date       daily_PnL adv          selected_strat
---------------------------------------------------
0  2019.01.02 32.81116  1.161895e+07 2             
1  2019.01.02 -17858.3  4.081003e+07 0             
2  2019.01.02 -5928.454 1.052341e+07 0             
3  2019.01.02 1129.518  6987232      0             
4  2019.01.02 1781.638  1.907262e+07 0             
5  2019.01.02 9802.122  2.222419e+07 2             
6  2019.01.02 2085.378  1.009195e+07 2             
7  2019.01.02 -26076.15 2.039854e+07 0             
8  2019.01.02 4382.17   2.821476e+07 2             
9  2019.01.02 -10117.18 1.895952e+07 2             
10 2019.01.02 -20978.37 5.362121e+07 0             
11 2019.01.02 -10575.18 1.371147e+07 2             
12 2019.01.02 -30436.72 4.953557e+07 2             
13 2019.01.02 3320.057  1.092683e+07 0             
14 2019.01.02 -2780.094 1.145875e+07 2             
15 2019.01.02 -7954.794 2.327105e+07 2             
16 2019.01.02 -4654.495 1.236677e+07 2             
17 2019.01.0

In [14]:
/show the average PnL of stocks of strat = 2
PnL_st1: select average_PnL: avg daily_PnL by date from tbl_full where selected_strat = 2
PnL_st1

date      | average_PnL
----------| -----------
2019.01.02| -14270.65  
2019.01.03| -28655.29  
2019.01.04| -23889.76  
2019.01.07| -18413.55  
2019.01.08| -19302.6   
2019.01.09| 3.992989   
2019.01.10| -12730.29  
2019.01.11| -12264.39  
2019.01.14| -11864.05  
2019.01.15| -11948.11  
2019.01.16| -10393.86  
2019.01.17| -8959.517  
2019.01.18| -14361.67  
2019.01.22| -19400.2   
2019.01.23| -11880.1   
2019.01.24| -17107.73  
2019.01.25| -15271.38  
2019.01.28| -14686.95  
2019.01.29| -12492.6   
2019.01.30| -12952.02  
..


In [15]:
/show the average PnL of stocks of strat = 0

PnL_st2: select average_PnL: avg daily_PnL by date from tbl_full where selected_strat = 0
PnL_st2

date      | average_PnL
----------| -----------
2019.01.02| -16744.16  
2019.01.03| -22859.36  
2019.01.04| -14187.68  
2019.01.07| -27797.47  
2019.01.08| -18030.32  
2019.01.09| 4.745304   
2019.01.10| -14515.66  
2019.01.11| -7695.718  
2019.01.14| -11261.56  
2019.01.15| -20893.57  
2019.01.16| -8351.607  
2019.01.17| -12147.43  
2019.01.18| -10713.14  
2019.01.22| -14757.21  
2019.01.23| -11216.59  
2019.01.24| -9805.661  
2019.01.25| -9851.242  
2019.01.28| -17720.03  
2019.01.29| -7874.248  
2019.01.30| -14688.03  
..


In [16]:
PnL_summary: select average_PnL: avg daily_PnL by selected_strat from tbl_full
PnL_summary

selected_strat| average_PnL
--------------| -----------
0             | -11789.78  
2             | -11163.8   


#### Show the data when we clean the 2019.01.09

In [17]:
select average_PnL: avg daily_PnL by selected_strat from tbl_full where date <> 2019.01.09

selected_strat| average_PnL
--------------| -----------
0             | -11986.23  
2             | -11354.17  


### 2.2

In [18]:
/update suitably normalized units PnL/adv
tbl_full: update norm_pnl: daily_PnL % adv from tbl_full
/compute the t test
PnL_summary1: select t_test: (avg norm_pnl) % dev norm_pnl by date, selected_strat from tbl_full
PnL_summary1

date       selected_strat| t_test    
-------------------------| ----------
2019.01.02 0             | -0.8921255
2019.01.02 2             | -0.9875145
2019.01.03 0             | -0.8941804
2019.01.03 2             | -0.900038 
2019.01.04 0             | -0.9415085
2019.01.04 2             | -1.030673 
2019.01.07 0             | -1.144546 
2019.01.07 2             | -1.035731 
2019.01.08 0             | -0.8962512
2019.01.08 2             | -0.8992952
2019.01.09 0             | 0.05719319
2019.01.09 2             | 0.1425226 
2019.01.10 0             | -0.6266972
2019.01.10 2             | -0.666487 
2019.01.11 0             | -0.8176585
2019.01.11 2             | -0.9808746
2019.01.14 0             | -0.7889831
2019.01.14 2             | -0.9578838
2019.01.15 0             | -0.89578  
2019.01.15 2             | -0.8112435
..


### 2.3

In [19]:
/update the month 
tbl_full: update month: `mm$date from tbl_full
tbl_full

id date       daily_PnL adv          selected_strat norm_pnl      month
-----------------------------------------------------------------------
0  2019.01.02 32.81116  1.161895e+07 2              2.823935e-06  1    
1  2019.01.02 -17858.3  4.081003e+07 0              -0.0004375959 1    
2  2019.01.02 -5928.454 1.052341e+07 0              -0.0005633588 1    
3  2019.01.02 1129.518  6987232      0              0.0001616546  1    
4  2019.01.02 1781.638  1.907262e+07 0              9.341337e-05  1    
5  2019.01.02 9802.122  2.222419e+07 2              0.0004410564  1    
6  2019.01.02 2085.378  1.009195e+07 2              0.0002066377  1    
7  2019.01.02 -26076.15 2.039854e+07 0              -0.001278335  1    
8  2019.01.02 4382.17   2.821476e+07 2              0.0001553148  1    
9  2019.01.02 -10117.18 1.895952e+07 2              -0.0005336199 1    
10 2019.01.02 -20978.37 5.362121e+07 0              -0.0003912326 1    
11 2019.01.02 -10575.18 1.371147e+07 2              -0.000771265

In [20]:
/here I just use the first 60 days data because of calculation speed
PnL_summary1: select t_test: (avg norm_pnl) % dev norm_pnl by month, selected_strat from tbl_full
PnL_summary1

month selected_strat| t_test    
--------------------| ----------
1     0             | -0.7780392
1     2             | -0.7703299
2     0             | -0.7752334
2     2             | -0.6969488
3     0             | -0.7631824
3     2             | -0.6482981


### 2.4

In [21]:
/set x as the input for calculating different prob
/the first element of x is date
/the second element of x is prob
computeFull2:{[x]
            d: x 0;
            prob1: x 1;
            raw: select from bin10 where date = d;
            raw: update reverse fills reverse date,
                reverse fills reverse mid,
                reverse fills reverse spread,
                reverse fills reverse vol,
                reverse fills reverse adv
                by id from raw;

            raw: update R: (neg 1) + (last mid) % mid by id from raw;
            raw: update vol_R: sqrt var deltas0 R by id from raw;
            raw: update W: vol_R * (0, sums u12[(count vol_R) - 1])  by id from raw;
            raw: raw cross ([]rho: 0, .05, .3; []name: 0, 1, 2);

            tbl: update alpha: createAlpha[rho;R; W] by id, rho from raw;
            tbl: update alpha_2: 0^((alpha - xprev[60; alpha]) % 10) by id, rho from tbl;

            tbl_impact: update I: computeImpact[alpha; alpha_2] by id, rho from tbl;
            tbl_impact: update I_p:  (1 xprev I) * (exp neg beta*dt) by id, rho from tbl_impact;

            tbl_dQ: update dQ: computeTrade[adv; vol; I; I_p] by id, rho from tbl_impact;

            tbl2: abTest[tbl_dQ; strat1; strat2; prob1];
            PnL_sec: computePnL[tbl2];
            PnL: 0!select first date, daily_PnL: sum PnL, first adv, first selected_strat by id from PnL_sec;
            PnL: update prob: prob1 from PnL;
            PnL}


In [22]:
/I only input the first 10 dates, beacuse running whole dates with each prob is too slow
prob_list :0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
new: cross[10#date_list; prob_list]
PnL2: raze computeFull2 each new
PnL2

id date       daily_PnL adv          selected_strat prob
--------------------------------------------------------
0  2019.01.02 -4445.331 1.161895e+07 0              0.1 
1  2019.01.02 -17075.9  4.081003e+07 0              0.1 
2  2019.01.02 -15271.89 1.052341e+07 0              0.1 
3  2019.01.02 -7485.654 6987232      0              0.1 
4  2019.01.02 -17021.23 1.907262e+07 0              0.1 
5  2019.01.02 -3912.196 2.222419e+07 0              0.1 
6  2019.01.02 -12532.16 1.009195e+07 2              0.1 
7  2019.01.02 -30791.05 2.039854e+07 0              0.1 
8  2019.01.02 -21595.43 2.821476e+07 0              0.1 
9  2019.01.02 3513.87   1.895952e+07 0              0.1 
10 2019.01.02 -40343.47 5.362121e+07 0              0.1 
11 2019.01.02 1412.019  1.371147e+07 0              0.1 
12 2019.01.02 -44875.18 4.953557e+07 0              0.1 
13 2019.01.02 -15508.71 1.092683e+07 0              0.1 
14 2019.01.02 -2597.777 1.145875e+07 0              0.1 
15 2019.01.02 -8289.884 2.32710

In [23]:
/ Daily PNL for AB TEST by different date
select avg daily_PnL by date from PnL2

date      | daily_PnL
----------| ---------
2019.01.02| -17841.59
2019.01.03| -25010.85
2019.01.04| -19945.66
2019.01.07| -17961.69
2019.01.08| -18066.03
2019.01.09| 0.9026281
2019.01.10| -13285.38
2019.01.11| -11595.83
2019.01.14| -10061.48
2019.01.15| -13101.77


In [24]:
/ Daily PNL for AB TEST by different prob
select avg daily_PnL by prob from PnL2

prob| daily_PnL
----| ---------
0.1 | -15523.03
0.2 | -14556.29
0.3 | -14188.46
0.4 | -14619.63
0.5 | -14474.95
0.6 | -14706.61
0.7 | -13973.6 
0.8 | -15956.97
0.9 | -14182.9 


In [25]:
/normalize and show t-test
PnL2: update norm_pnl: daily_PnL % adv from PnL2
select t_test: (avg norm_pnl) % dev norm_pnl by date, selected_strat, prob from PnL2

date       selected_strat prob| t_test    
------------------------------| ----------
2019.01.02 0              0.1 | -0.8994509
2019.01.02 0              0.2 | -0.8958165
2019.01.02 0              0.3 | -0.8808583
2019.01.02 0              0.4 | -0.9317917
2019.01.02 0              0.5 | -0.779415 
2019.01.02 0              0.6 | -0.9600456
2019.01.02 0              0.7 | -0.9026048
2019.01.02 0              0.8 | -0.9454451
2019.01.02 0              0.9 | -0.8800735
2019.01.02 2              0.1 | -1.376666 
2019.01.02 2              0.2 | -1.098733 
2019.01.02 2              0.3 | -0.8816674
2019.01.02 2              0.4 | -1.093655 
2019.01.02 2              0.5 | -0.9117851
2019.01.02 2              0.6 | -0.7869951
2019.01.02 2              0.7 | -0.9149238
2019.01.02 2              0.8 | -0.92538  
2019.01.02 2              0.9 | -0.8820795
2019.01.03 0              0.1 | -0.8580996
2019.01.03 0              0.2 | -0.8789021
..


Comment : when using A&B test, a higher PnL is an important indicator of better performance. Also, we want to ensure that the reason of difference in PnL is randomness. Thus both PnL and uncertainty are our important indicator, so there is a trade-off.

### 2.5
#### PnL for strat 1 and 0 
#### similar process like 2.4, but in each function, I change the name of strat1 and strat2

In [26]:
computeFull2_10:{[x]
            d: x 0;
            prob1: x 1;
            raw: select from bin10 where date = d;
            raw: update reverse fills reverse date,
                reverse fills reverse mid,
                reverse fills reverse spread,
                reverse fills reverse vol,
                reverse fills reverse adv
                by id from raw;

            raw: update R: (neg 1) + (last mid) % mid by id from raw;
            raw: update vol_R: sqrt var deltas0 R by id from raw;
            raw: update W: vol_R * (0, sums u12[(count vol_R) - 1])  by id from raw;
            raw: raw cross ([]rho: 0, .05, .3; []name: 0, 1, 2);

            tbl: update alpha: createAlpha[rho;R; W] by id, rho from raw;
            tbl: update alpha_2: 0^((alpha - xprev[60; alpha]) % 10) by id, rho from tbl;

            tbl_impact: update I: computeImpact[alpha; alpha_2] by id, rho from tbl;
            tbl_impact: update I_p:  (1 xprev I) * (exp neg beta*dt) by id, rho from tbl_impact;

            tbl_dQ: update dQ: computeTrade[adv; vol; I; I_p] by id, rho from tbl_impact;

            tbl2: abTest[tbl_dQ; strat1:1; strat2:0; prob1];
            PnL_sec: computePnL[tbl2];
            PnL: 0!select first date, daily_PnL: sum PnL, first adv, first selected_strat by id from PnL_sec;
            PnL: update prob: prob1 from PnL;
            PnL}


In [27]:
PnL2_10: raze computeFull2_10 each new
PnL2_10

id date       daily_PnL adv          selected_strat prob
--------------------------------------------------------
0  2019.01.02 -15493.54 1.161895e+07 1              0.1 
1  2019.01.02 -19158.37 4.081003e+07 1              0.1 
2  2019.01.02 -6408.551 1.052341e+07 0              0.1 
3  2019.01.02 -2332.197 6987232      1              0.1 
4  2019.01.02 -7335.498 1.907262e+07 0              0.1 
5  2019.01.02 6406.381  2.222419e+07 0              0.1 
6  2019.01.02 -5210.467 1.009195e+07 0              0.1 
7  2019.01.02 1713.392  2.039854e+07 0              0.1 
8  2019.01.02 -43792.11 2.821476e+07 0              0.1 
9  2019.01.02 -37791.08 1.895952e+07 0              0.1 
10 2019.01.02 -30473.7  5.362121e+07 0              0.1 
11 2019.01.02 -16061.91 1.371147e+07 0              0.1 
12 2019.01.02 -33756.03 4.953557e+07 1              0.1 
13 2019.01.02 -24991.02 1.092683e+07 0              0.1 
14 2019.01.02 -5655.198 1.145875e+07 0              0.1 
15 2019.01.02 -25675.76 2.32710

In [37]:
/ Daily PNL for AB TEST by different date
select avg daily_PnL by date, selected_strat from PnL2_10

date       selected_strat| daily_PnL
-------------------------| ---------
2019.01.02 0             | -18619.24
2019.01.02 1             | -17676.96
2019.01.03 0             | -22753.75
2019.01.03 1             | -21419.29
2019.01.04 0             | -21296.14
2019.01.04 1             | -20281.22
2019.01.07 0             | -18262.45
2019.01.07 1             | -17990.8 
2019.01.08 0             | -17624.23
2019.01.08 1             | -15888.83
2019.01.09 0             | 1.760901 
2019.01.09 1             | 1.239092 
2019.01.10 0             | -13113.26
2019.01.10 1             | -14478.87
2019.01.11 0             | -11227.39
2019.01.11 1             | -11376.5 
2019.01.14 0             | -10431.72
2019.01.14 1             | -11681.46
2019.01.15 0             | -13037.96
2019.01.15 1             | -13138.64


In [38]:
/ Daily PNL for AB TEST by different prob
select avg daily_PnL by prob, selected_strat from PnL2_10

prob selected_strat| daily_PnL
-------------------| ---------
0.1  0             | -15138.87
0.1  1             | -13907.94
0.2  0             | -13655.87
0.2  1             | -14537.28
0.3  0             | -13781.16
0.3  1             | -14473.59
0.4  0             | -14890.95
0.4  1             | -13519.2 
0.5  0             | -15061.71
0.5  1             | -13884.92
0.6  0             | -14733.66
0.6  1             | -15933.08
0.7  0             | -15428.7 
0.7  1             | -15590.4 
0.8  0             | -15940.8 
0.8  1             | -13333.16
0.9  0             | -15169.16
0.9  1             | -14031.19


In [39]:
/normalize and t-test
PnL2_10: update norm_pnl: daily_PnL % adv from PnL2_10
select t_test: (avg norm_pnl) % dev norm_pnl by date, selected_strat, prob from PnL2_10

date       selected_strat prob| t_test    
------------------------------| ----------
2019.01.02 0              0.1 | -0.968187 
2019.01.02 0              0.2 | -0.9638845
2019.01.02 0              0.3 | -0.9156061
2019.01.02 0              0.4 | -1.035783 
2019.01.02 0              0.5 | -0.842138 
2019.01.02 0              0.6 | -0.9815534
2019.01.02 0              0.7 | -0.9481269
2019.01.02 0              0.8 | -0.8573081
2019.01.02 0              0.9 | -0.7570318
2019.01.02 1              0.1 | -0.8126294
2019.01.02 1              0.2 | -1.008909 
2019.01.02 1              0.3 | -0.8664911
2019.01.02 1              0.4 | -0.9736884
2019.01.02 1              0.5 | -1.063558 
2019.01.02 1              0.6 | -0.8325365
2019.01.02 1              0.7 | -0.8692316
2019.01.02 1              0.8 | -0.9168843
2019.01.02 1              0.9 | -0.9815104
2019.01.03 0              0.1 | -0.8889615
2019.01.03 0              0.2 | -0.8237688
..


#### PnL for strat 1 and 2

In [31]:
computeFull2_12:{[x]
            d: x 0;
            prob1: x 1;
            raw: select from bin10 where date = d;
            raw: update reverse fills reverse date,
                reverse fills reverse mid,
                reverse fills reverse spread,
                reverse fills reverse vol,
                reverse fills reverse adv
                by id from raw;

            raw: update R: (neg 1) + (last mid) % mid by id from raw;
            raw: update vol_R: sqrt var deltas0 R by id from raw;
            raw: update W: vol_R * (0, sums u12[(count vol_R) - 1])  by id from raw;
            raw: raw cross ([]rho: 0, .05, .3; []name: 0, 1, 2);

            tbl: update alpha: createAlpha[rho;R; W] by id, rho from raw;
            tbl: update alpha_2: 0^((alpha - xprev[60; alpha]) % 10) by id, rho from tbl;

            tbl_impact: update I: computeImpact[alpha; alpha_2] by id, rho from tbl;
            tbl_impact: update I_p:  (1 xprev I) * (exp neg beta*dt) by id, rho from tbl_impact;

            tbl_dQ: update dQ: computeTrade[adv; vol; I; I_p] by id, rho from tbl_impact;

            tbl2: abTest[tbl_dQ; strat1:1; strat2:2; prob1];
            PnL_sec: computePnL[tbl2];
            PnL: 0!select first date, daily_PnL: sum PnL, first adv, first selected_strat by id from PnL_sec;
            PnL: update prob: prob1 from PnL;
            PnL}


In [32]:
PnL2_12: raze computeFull2_12 each new
PnL2_12

id date       daily_PnL adv          selected_strat prob
--------------------------------------------------------
0  2019.01.02 -8052.386 1.161895e+07 2              0.1 
1  2019.01.02 -41150.02 4.081003e+07 2              0.1 
2  2019.01.02 -12784.53 1.052341e+07 2              0.1 
3  2019.01.02 -9812.002 6987232      2              0.1 
4  2019.01.02 -19967.42 1.907262e+07 2              0.1 
5  2019.01.02 -112.1735 2.222419e+07 2              0.1 
6  2019.01.02 -5541.945 1.009195e+07 2              0.1 
7  2019.01.02 -8283.414 2.039854e+07 2              0.1 
8  2019.01.02 -35727.51 2.821476e+07 2              0.1 
9  2019.01.02 -41982.3  1.895952e+07 2              0.1 
10 2019.01.02 -41862.76 5.362121e+07 2              0.1 
11 2019.01.02 -29544.35 1.371147e+07 2              0.1 
12 2019.01.02 -48717.87 4.953557e+07 2              0.1 
13 2019.01.02 -25444.78 1.092683e+07 2              0.1 
14 2019.01.02 -3680.278 1.145875e+07 2              0.1 
15 2019.01.02 -11280.83 2.32710

In [40]:
/ Daily PNL for AB TEST by different date
select avg daily_PnL by date, selected_strat from PnL2_12

date       selected_strat| daily_PnL
-------------------------| ---------
2019.01.02 1             | -18476.67
2019.01.02 2             | -18636.8 
2019.01.03 1             | -25317.28
2019.01.03 2             | -24646.95
2019.01.04 1             | -20693.6 
2019.01.04 2             | -20605.66
2019.01.07 1             | -17032.38
2019.01.07 2             | -19396.39
2019.01.08 1             | -17934.08
2019.01.08 2             | -18335.74
2019.01.09 1             | 1.809597 
2019.01.09 2             | -2.895723
2019.01.10 1             | -13434.35
2019.01.10 2             | -12593.56
2019.01.11 1             | -11663.98
2019.01.11 2             | -12866.13
2019.01.14 1             | -10679.8 
2019.01.14 2             | -11534.53
2019.01.15 1             | -13713.19
2019.01.15 2             | -13640.64


In [41]:
/ Daily PNL for AB TEST by different prob
select avg daily_PnL by prob, selected_strat from PnL2_12

prob selected_strat| daily_PnL
-------------------| ---------
0.1  1             | -10368.1 
0.1  2             | -15781.63
0.2  1             | -15901.88
0.2  2             | -14842.1 
0.3  1             | -16332.6 
0.3  2             | -14951.97
0.4  1             | -14104.9 
0.4  2             | -15823.41
0.5  1             | -13260.49
0.5  2             | -15162.06
0.6  1             | -15010.55
0.6  2             | -15783.04
0.7  1             | -13522.27
0.7  2             | -15894.79
0.8  1             | -15107.34
0.8  2             | -12821.9 
0.9  1             | -16733.01
0.9  2             | -13453.31


In [42]:
/normalize and t-test
PnL2_12: update norm_pnl: daily_PnL % adv from PnL2_12
select t_test: (avg norm_pnl) % dev norm_pnl by date, selected_strat, prob from PnL2_12

date       selected_strat prob| t_test    
------------------------------| ----------
2019.01.02 1              0.1 | -1.212001 
2019.01.02 1              0.2 | -1.025471 
2019.01.02 1              0.3 | -0.88629  
2019.01.02 1              0.4 | -0.9791942
2019.01.02 1              0.5 | -0.8984949
2019.01.02 1              0.6 | -0.9206355
2019.01.02 1              0.7 | -1.054573 
2019.01.02 1              0.8 | -0.8476881
2019.01.02 1              0.9 | -0.8898475
2019.01.02 2              0.1 | -0.9934356
2019.01.02 2              0.2 | -0.919285 
2019.01.02 2              0.3 | -0.927138 
2019.01.02 2              0.4 | -0.9553886
2019.01.02 2              0.5 | -1.001698 
2019.01.02 2              0.6 | -1.083277 
2019.01.02 2              0.7 | -1.036394 
2019.01.02 2              0.8 | -1.107871 
2019.01.02 2              0.9 | -0.8816891
2019.01.03 1              0.1 | -0.8753394
2019.01.03 1              0.2 | -0.8947444
..


Waiting time refers to the duration of experiments that is needed for a trader to tell one strategy is better than the other. Thus, as t-stats become bigger, the waiting time become lower beacuse of fewer samples which resulting in better alpha.